# 回顾自动化机器学习解释

随着机器学习越来越普遍，模型做出的预测对我们社会的许多方面产生了更大的影响。例如，机器学习模型在银行如何决定发放贷款或医生如何确定治疗优先顺序方面成为了一个愈发重要的因素。解释和说明模型的功能变得越来越重要，以便可以说明和证明机器学习模型做出预测的基本原理，并且可以确定模型中任何无意的偏差。

使用自动化机器学习来训练模型时，可以选择生成特征重要性的解释，以量化每个特征对标签预测的影响程度。在本实验室中，你将探索由自动化机器学习试验生成的解释。

## 连接到工作区

你首先需要使用 Azure ML SDK 连接到工作区。

> **备注**：如果 Azure 订阅的身份验证会话在你完成上一练习后已过期，系统将提示你重新进行身份验证。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 从保存的配置文件加载工作区
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 准备计算目标

你将使用在前面的实验室中创建的 Azure 机器学习计算群集（如果不存在，则将创建它）。

> **注意事项**：在运行以下代码之前，请先在代码中将 *“你的计算群集”* 更改为你的计算群集的名称！群集名称必须是长度在 2 到 16 个字符之间的全局唯一名称。有效字符是字母、数字和 - 字符。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # 检查现有的计算目标
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # 如果尚不存在，请创建它
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

## 运行自动化机器学习试验

在本实验室中，为节省时间，你将运行自动化机器学习试验，且只有三次迭代。

请注意，将 **model_explainability** 配置选项设为 **True**。

In [ ]:
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails
from azureml.core import Dataset

try:
    # 准备训练数据
    default_ds = ws.get_default_datastore()
    if 'diabetes dataset' not in ws.datasets:
        default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # 将糖尿病 csv 文件上传到 /data 中
                            target_path='diabetes-data/', # 将其放在数据存储的文件夹路径中
                            overwrite=True, # 替换名称相同的现有文件
                            show_progress=True)

        # 从数据存储上的路径创建表格数据集（这可能需要一些时间）
        tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

        # 注册表格数据集
        try:
            tab_data_set = tab_data_set.register(workspace=ws, 
                                    name='diabetes dataset',
                                    description='diabetes data',
                                    tags = {'format':'CSV'},
                                    create_new_version=True)
            print('Dataset registered.')
        except Exception as ex:
            print(ex)
    else:
        print('Dataset already registered.')
    train_data = ws.datasets.get("diabetes dataset")


    # 配置自动化 ML
    automl_config = AutoMLConfig(name='Automated ML Experiment',
                                task='classification',
                                compute_target=training_cluster,
                                training_data = train_data,
                                n_cross_validations = 2,
                                label_column_name='Diabetic',
                                iterations=3,
                                primary_metric = 'AUC_weighted',
                                max_concurrent_iterations=3,
                                featurization='off',
                                model_explainability=True # 生成特征重要性！
                                )

    # 运行自动化 ML 试验
    print('Submitting Auto ML experiment...')
    automl_experiment = Experiment(ws, 'diabetes_automl')
    automl_run = automl_experiment.submit(automl_config)
    automl_run.wait_for_completion(show_output=True)
    RunDetails(automl_run).show()

except Exception as ex:
    print(ex)

## 查看特征重要性

完成试验后，在上面的小组件中，单击生成最佳结果的运行以查看其详细信息。然后滚动到可视化效果底部以查看相对特征重要性。


## 在 Azure 机器学习工作室中查看模型解释

完成试验运行后，单击小组件中的链接以在 Azure 机器学习工作室中查看运行，然后查看 **“解释”** 选项卡。然后：

1. 选择由自动化机器学习运行创建的解释器。
2. 查看 **“全局重要性”** 图表，其中显示整体全局特征重要性。
3. 查看 **“摘要重要性”** 图表，其中显示分簇散点图、*小提琴* *图* 或 *盒* 须图中测试数据的每个数据点。
4. 选择单个点以查看所选数据点的单个预测的 **“局部特征重要性”**。

> **更多信息**：有关自动化机器学习的详细信息，请参阅 [Azure ML 文档](https://docs.microsoft.com/azure/machine-learning/how-to-machine-learning-interpretability-automl)。